In [2]:
from typing import AnyStr
import ast
string = "s = 5\nmyString = 'Hell'\nprint(myString)"
tree = compile(source = string, filename ="test", mode="exec", flags= ast.PyCF_ONLY_AST)

In [3]:
from graphviz import Digraph
dot = Digraph()
# Define a function to recursively add nodes to the Digraph
def add_node(node, parent=None):
    node_name = str(node.__class__.__name__)
    dot.node(str(id(node)), node_name)
    if parent:
        dot.edge(str(id(parent)), str(id(node)))
    for child in ast.iter_child_nodes(node):
        add_node(child, node)


# Add nodes to the Digraph
add_node(tree)

# Render the Digraph as a PNG file
dot.format = 'png'
dot.render('my_ast', view=True)

'my_ast.png'

In [4]:
class v(ast.NodeVisitor):
   def generic_visit(self, node):
    print(type(node).__name__)
    m = ast.NodeVisitor.generic_visit(self, node)

n = v()
# print(n.visit(tree))

In [5]:
# import ast
# my_ast:ast = ast.parse(source=string)

# # def ast_visit_v1():


# def getHeight(node, height=0):
#     max = height
#     for n in ast.iter_child_nodes(node):
#         retHeight = getHeight(n, height=height+1)
#         max = retHeight if retHeight > max else max
#     return max

# height = getHeight(my_ast)
# print(height)

In [6]:
import ast
def str_node(node):
    if isinstance(node, ast.AST):
        fields = [(name, str_node(val)) for name, val in ast.iter_fields(node)]
        rv = '%s(%s' % (node.__class__.__name__, ', '.join('%s=%s' % field for field in fields))
        return rv + ')'
    else:
        return repr(node)
def ast_visit(node, level=0):
    print('  ' * level + str_node(node))
    for field, value in ast.iter_fields(node):
        if isinstance(value, list):
            for item in value:
                if isinstance(item, ast.AST):
                    ast_visit(item, level=level+1)
        elif isinstance(value, ast.AST):
            ast_visit(value, level=level+1)


ast_visit(ast.parse('x = a + b'))

Module(body=[<ast.Assign object at 0x000001B69238FCD0>], type_ignores=[])
  Assign(targets=[<ast.Name object at 0x000001B69238FD30>], value=BinOp(left=Name(id='a', ctx=Load()), op=Add(), right=Name(id='b', ctx=Load())), type_comment=None)
    Name(id='x', ctx=Store())
      Store()
    BinOp(left=Name(id='a', ctx=Load()), op=Add(), right=Name(id='b', ctx=Load()))
      Name(id='a', ctx=Load())
        Load()
      Add()
      Name(id='b', ctx=Load())
        Load()


In [7]:
def removing_elements(my_list, element, index=0):
   result = [i for i in my_list if i[index] != element]
   return result

In [8]:
# implementing gumTree
import itertools
from utilities_parsing import *
# raise Exception("stop")
T1 = tree
T2 = tree
""" parentify and addHeightAttribute are utility functions that are used to add parent and height attributes to the nodes of the AST. """
parentify(T1); parentify(T2)
addHeightAttribute(T1); addHeightAttribute(T2)
""" Initialize the priority queues L1 and L2 with the root nodes of T1 and T2, respectively. """
L1 = PriorityQueue()
L2 = PriorityQueue()
A = list()
M = set()
isomorphism = {}
L1.put((getHeight(T1), id(T1), T1))
L2.put((getHeight(T2), id(T2), T2))
# L1Size = 1
# L2Size = 1
# minHeight = 2
isomorphic = compare_ast
def topdown(T1, T2, L1, L2, A, M, minHeight = 2, L1Size=1, L2Size=1):
    # peekMax = getHeight  #peekMax is the maximum height but I do not know why literature names it like this
    while (min(peekMax(L1),peekMax(L2)) > minHeight):
        if (peekMax(L1) != peekMax(L2)):
            if (peekMax(L1) > peekMax(L2)):
                # pop all elements having a height equal to peekMax
                poppedT = popUtility(L1)
                L1Size = L1Size - len(poppedT)
                for t in poppedT:
                    for child in ast.iter_child_nodes(t):
                        L1.put((getHeight(child), id(child), child))
                        L1Size += 1
            else:
                poppedT = popUtility(L2)
                L2Size = L2Size - len(poppedT)

                for t in poppedT:
                    for child in ast.iter_child_nodes(t):
                        L2.put((getHeight(child), id(child), child))
                        L2Size += 1
        else:
            H1 = popUtility(L1)
            H2 = popUtility(L2)
            H1CrossH2 = list()
            H1CrossH2.append(H1)
            H1CrossH2.append(H2)
            H1CrossH2 = list(H1CrossH2)
            for t1, t2 in itertools.product(*H1CrossH2):
                if isomorphic(t1, t2):
                    thereExists = checkOtherT(t1, t2, T2)
                    if (not thereExists): thereExists = checkOtherT(t2, t1, T1)
                    if (thereExists):
                        A.append((t1, t2))
                    else:
                        l = list(zip(ast.walk(t1), ast.walk(t2))) # add all pairs of isomorphic nodes to 
                        # ignore the first pair as it is the node itself
                        i = 0
                        for t11, t22 in (l):
                            if (i == 0): i+=1; continue
                            M.add((t11, t22))
                        # print("exists")
            # for m, n in M:
            #     print(m, n)
            AunionM = set(A) | M
            # print(AunionM)
            for t1 in H1:
                notInUnion = True
                for t11, t22 in AunionM:
                    if t11 is t1:
                        notInUnion = False
                        break
                if (notInUnion):
                    for child in ast.iter_child_nodes(t1):
                        L1.put((getHeight(t1), id(t1), t1))
                        L1Size = L1Size + 1
            
            for t2 in H2:
                notInUnion = True
                for t11, t22 in AunionM:
                    if t22 is t2: 
                        notInUnion = False
                        break
                if (notInUnion): #put its descendants in the queue
                    for child in ast.iter_child_nodes(t2):
                        L2.put((getHeight(child), id(child), child))
                        L2Size += 1
    A = sorted(A, key=lambda x: dice(x[0], x[1], M), reverse=True)
    while (len(A) > 0):
        t1, t2 = A.pop(0)
        # add all pairs of isomorphic nodes to M
        l = list(zip(ast.walk(t1), ast.walk(t2)))
        i = 0
        for t11, t22 in (l):
            if (i == 0): i+=1; continue
            M.add((t11, t22))
        removing_elements(A, t1, 0)
        removing_elements(A, t2, 1)
            

topdown(T1, T2, L1, L2, A, M)


# print(A)

In [9]:
def has_matched_children(node1, node2, M):
    for child1 in ast.iter_child_nodes(node1):
        for t1, t2 in M:
            if (t1 is child1):
                return True
    return False

In [10]:
def bottomUp(T1, T2, M, minDice = 0.5, minHeight = 2):
    T1 = postordertraversal(t1)
    for t1 in T1:
        if (not is_leaf_node(t1) and has_matched_children(t1, t2, M)):
            for t2 in T2:
                if (not is_leaf_node(t2) and has_matched_children(t2, t1, M)):
                    if (dice(t1, t2, M) >= minDice):
                        M.add((t1, t2))
    pass

In [11]:
from textwrap import indent


def traversePostorder(node):
    for child in ast.iter_child_nodes(node):
        traversePostorder(child)
    print(node)

print(ast.dump(tree , indent=4))

Module(
    body=[
        Assign(
            targets=[
                Name(id='s', ctx=Store())],
            value=Constant(value=5)),
        Assign(
            targets=[
                Name(id='myString', ctx=Store())],
            value=Constant(value='Hell')),
        Expr(
            value=Call(
                func=Name(id='print', ctx=Load()),
                args=[
                    Name(id='myString', ctx=Load())],
                keywords=[]))],
    type_ignores=[])


In [12]:
import apted
# print(T1)
# like = apted.APTED(T1, T2, 1, 1)

In [13]:
def add_node(node, parent=None):
    node_name = str(node.__class__.__name__)
    dot.node(str(id(node)), node_name)
    if parent:
        dot.edge(str(id(parent)), str(id(node)))
    for child in ast.iter_child_nodes(node):
        add_node(child, node)

In [14]:
a = ast.parse("x = 5")
b = ast.parse("y = 5")
print(ast.dump(a, indent = 4))
print(a.__class__.__name__)
if (a == b): print("p")

Module(
    body=[
        Assign(
            targets=[
                Name(id='x', ctx=Store())],
            value=Constant(value=5))],
    type_ignores=[])
Module


In [15]:
import ast

class FunctionCallVisitor(ast.NodeVisitor):
    def visit_Call(self, node):
        if isinstance(node.func, ast.Name) and node.func.id == "print":
            args = [arg for arg in node.args if isinstance(arg, ast.Constant)]
            if args:
                print("Detected print statements with string literals:")
                for arg in args:
                    print(arg.s)  # Print the string literal directly
        self.generic_visit(node)

In [16]:
def perform_static_analysis(code):
    tree = ast.parse(code)
    visitor = FunctionCallVisitor()
    visitor.visit(tree)

In [17]:
code = "h = 'pipp'; print('pola')"
perform_static_analysis(code)

Detected print statements with string literals:
pola


In [18]:
import itertools

somelists = [
   [1],
   ['a']]
for element, e in itertools.product(*somelists):
    print(element)

1


In [19]:
l = [1, 2 , 3]
l2 = [3, 4, 4]
print(list(zip(l, l2)))

[(1, 3), (2, 4), (3, 4)]


In [20]:
import ast
import astunparse

expr="""
s = 5
"""
p=ast.parse(expr)
p.hegiht = 2
# print(astunparse.unparse(p))


# p.body[0].body = [ ast.parse("return 42")] # Replace function body with "return 42"


for t1, t2 in M:
    if (type(t1) == ast.Store or type(t1) == ast.Load): continue
    print(astunparse.unparse(t1).replace("\n", ""))
    print(astunparse.unparse(t2).replace("\n", ""))




myString
myString
print(myString)
print(myString)
myString
myString
print
print
s
s
'Hell'
'Hell'
5
5
s = 5
s = 5
myString = 'Hell'
myString = 'Hell'
print(myString)
print(myString)


In [21]:
import ast

""" What do you want here?"""

try:
    res = ast.parse("x = 5")
except SyntaxError as e:
    print(e)
    print(e.text)
    print(e.msg)
# def get_ast(node):
#     for k , f in ast.iter_fields(res):
#         print(k, f)
#         get_ast(f)
# get_ast(res)
    
def descendants(node):
        return [n for n in ast.walk(node)][1:]

def postordertraversal(node, listNodes):
    for child in ast.iter_child_nodes(node):
        postordertraversal(child, listNodes)
    listNodes.append(node)

listNodes = []
postordertraversal(res, listNodes)

for node in listNodes:
    print(node.__class__.__name__)



Store
Name
Constant
Assign
Module


In [22]:
codesearch='<bos><start_of_turn>user\nYou are a python expert and your task is: Given the following description and python code:\n  Description:\n      Check if in given list of numbers, are any two numbers closer to each other than\n    given threshold.\n    >>> has_close_elements([1.0, 2.0, 3.0], 0.5)\n    False\n    >>> has_close_elements([1.0, 2.8, 3.0, 4.0, 5.0, 2.0], 0.3)\n    True\n  Code:\n  \'\'\'python\n  from typing import List\n\n\ndef has_close_elements(numbers: List[float], threshold: float) -> bool:\n        for idx, elem in enumerate(numbers):\n        for idx2, elem2 in enumerate(numbers):\n            if idx != idx2:\n                distance = abs(elem - elem2)\n                if distance < threshold:\n                    return True\n\n    return False\n\'\'\'\n  Generate a class that contains at least 7 unit tests (where each test has only one assertion) written in python that acheive high coverage to find bugs, runtime errors or logical errors in the code to conform with the description and include any required imports.\n  You are given examples of unit tests for a similar code, which you can use to write the unit tests for the given code.:\n  \'\'\'python\n  \n\nMETADATA = {\n    \'author\': \'jt\',\n    \'dataset\': \'test\'\n}\n\n\ndef check(candidate):\n    assert candidate([]) == []\n    assert candidate([1, 2, 3, 4]) == [1, 2, 3, 4]\n    assert candidate([1, 2, 3, 2, 4, 3, 5]) == [1, 4, 5]\n\'\'\'\n  Make sure to include the unit test call __main__ to run the tests.\n  The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```python" and "```":<end_of_turn>\n<start_of_turn>model\n```python\n\nimport unittest\n\n\nclass TestHasCloseElements(unittest.TestCase):\n\n    def setUp(self):\n        self.numbers = None\n\n    def tearDown(self):\n        self.numbers = None\n\n    def test_empty_list(self):\n        self.assertEqual(has_close_elements([], 0.5), False)\n\n    def test_simple_list(self):\n        self.assertEqual(has_close_elements([1, 2, 3, 4], 0.5), False)\n\n    def test_multiple_close_elements(self):\n        self.assertEqual(has_close_elements([1, 2, 3, 2, 4, 3, 5], 0.3), True)\n\n    def test_threshold_too_high(self):\n        self.assertEqual(has_close_elements([1, 2, 3, 4], 1.0), False)\n\n    def test_different_threshold(self):\n        self.assertEqual(has_close_elements([1, 2, 3, 4], 0.2), False)\n\n    def test_duplicates(self):\n        self.assertEqual(has_close_elements([1, 2, 3, 2, 4, 3, 5], 0.3), [1, 2, 3, 4])\n\n\nif __name__ == \'__main__\':\n    unittest.main()\n```\n\nThis code snippet includes the required imports, defines a test class called `TestHasCloseElements` that contains 7 unit tests, and includes the `__main__` call to run the tests. Each test has only one assertion, which helps achieve high coverage and makes it easier to find bugs.'

In [23]:
import ast
import re


# codesearch = """from typing import List, Any def filter_integers(values: List[Any]) -> List[int]: \"\"\" Filter given list of any python values only for integers >>> filter_integers(['a', 3.14, 5]) [5] >>> filter_integers([1, 2, 3, 'abc', {}, []]) [1, 2, 3] \"\"\""""
code = re.search(r"[^\"](?<=```python\n)(.*)(?=```)", codesearch, re.DOTALL)
print(code.group(0))


# code = """
# for idx, elem in enumerate(numbers):
#         for idx2, elem2 in enumerate(numbers):
#             if idx != idx2:
#                 distance = abs(elem - elem2)
#                 if distance < threshold:
#                     return True
#     return False"""

# res = ast.parse(code)
# print(ast.dump(res))




import unittest


class TestHasCloseElements(unittest.TestCase):

    def setUp(self):
        self.numbers = None

    def tearDown(self):
        self.numbers = None

    def test_empty_list(self):
        self.assertEqual(has_close_elements([], 0.5), False)

    def test_simple_list(self):
        self.assertEqual(has_close_elements([1, 2, 3, 4], 0.5), False)

    def test_multiple_close_elements(self):
        self.assertEqual(has_close_elements([1, 2, 3, 2, 4, 3, 5], 0.3), True)

    def test_threshold_too_high(self):
        self.assertEqual(has_close_elements([1, 2, 3, 4], 1.0), False)

    def test_different_threshold(self):
        self.assertEqual(has_close_elements([1, 2, 3, 4], 0.2), False)

    def test_duplicates(self):
        self.assertEqual(has_close_elements([1, 2, 3, 2, 4, 3, 5], 0.3), [1, 2, 3, 4])


if __name__ == '__main__':
    unittest.main()

